# Data Cleaning Note Book for Ed Team

## Import Dependencies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

### Read in Data


In [2]:
data = pd.read_csv('Cafe_sales.csv')

## Cleaning Methods

### Find and Replace Outliers

In [3]:
df_fixed = data
# Import needed libraries for identifying outliers
import numpy as np

# Function to calculate and replace outliers using the 1.5 × IQR rule
def replace_outliers_with_nan(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = df[col].where((df[col] >= lower_bound) & (df[col] <= upper_bound), np.nan)

# Fixing outliers in 'Total Spent'
replace_outliers_with_nan(df_fixed, 'Total Spent')

### Find and Replace Null Values

In [4]:
# Fill missing values with appropriate strategies for non-numeric columns
df_fixed['Transaction ID'] = df_fixed['Transaction ID'].fillna('Unknown')
df_fixed['Item'] = df_fixed['Item'].fillna('Unknown')
df_fixed['Payment Method'] = df_fixed['Payment Method'].fillna('Unknown')
df_fixed['Location'] = df_fixed['Location'].fillna('Unknown')
df_fixed['Transaction Date'] = df_fixed['Transaction Date'].fillna('Unknown')

# Calculate missing values from the other two columns
# If Total Spent is missing but we have Quantity and Price Per Unit
mask = df_fixed['Total Spent'].isna() & df_fixed['Quantity'].notna() & df_fixed['Price Per Unit'].notna()
df_fixed.loc[mask, 'Total Spent'] = df_fixed.loc[mask, 'Quantity'] * df_fixed.loc[mask, 'Price Per Unit']

# If Price Per Unit is missing but we have Total Spent and Quantity
mask = df_fixed['Price Per Unit'].isna() & df_fixed['Total Spent'].notna() & df_fixed['Quantity'].notna()
df_fixed.loc[mask, 'Price Per Unit'] = df_fixed.loc[mask, 'Total Spent'] / df_fixed.loc[mask, 'Quantity']

# If Quantity is missing but we have Total Spent and Price Per Unit
mask = df_fixed['Quantity'].isna() & df_fixed['Total Spent'].notna() & df_fixed['Price Per Unit'].notna()
df_fixed.loc[mask, 'Quantity'] = df_fixed.loc[mask, 'Total Spent'] / df_fixed.loc[mask, 'Price Per Unit']

# For any remaining missing values, use the mean as fallback
df_fixed['Quantity'] = df_fixed['Quantity'].fillna(df_fixed['Quantity'].mean())
df_fixed['Price Per Unit'] = df_fixed['Price Per Unit'].fillna(df_fixed['Price Per Unit'].mean())
df_fixed['Total Spent'] = df_fixed['Total Spent'].fillna(df_fixed['Total Spent'].mean())

# Round money columns to 2 decimal places
df_fixed['Price Per Unit'] = df_fixed['Price Per Unit'].round(2)
df_fixed['Total Spent'] = df_fixed['Total Spent'].round(2)

In [5]:
df_fixed

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,08-09-2023
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,16-05-2023
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,19-07-2023
3,TXN_7034554,Salad,2.0,5.0,10.0,Unknown,Unknown,27-04-2023
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,11-06-2023
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2.0,2.0,4.0,Unknown,Unknown,30-08-2023
9996,TXN_9659401,Unknown,3.0,1.0,3.0,Digital Wallet,Unknown,02-06-2023
9997,TXN_5255387,Coffee,4.0,2.0,8.0,Digital Wallet,Unknown,02-03-2023
9998,TXN_7695629,Cookie,3.0,1.0,3.0,Digital Wallet,Unknown,02-12-2023


In [6]:
def no_duplicates(dataframe = df_fixed, column = "Transaction ID"):
    df_clean = dataframe.drop_duplicates(subset=column)
    return df_clean

df_no_dupes = no_duplicates()
df_no_dupes

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,08-09-2023
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,16-05-2023
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,19-07-2023
3,TXN_7034554,Salad,2.0,5.0,10.0,Unknown,Unknown,27-04-2023
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,11-06-2023
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2.0,2.0,4.0,Unknown,Unknown,30-08-2023
9996,TXN_9659401,Unknown,3.0,1.0,3.0,Digital Wallet,Unknown,02-06-2023
9997,TXN_5255387,Coffee,4.0,2.0,8.0,Digital Wallet,Unknown,02-03-2023
9998,TXN_7695629,Cookie,3.0,1.0,3.0,Digital Wallet,Unknown,02-12-2023


## Write Data to New CSV File

In [7]:
# Write to CSV
df_fixed.to_csv('cleaned_cafe_data.csv', index=False)